# FNmodel Seed Runner Experiment
FNmodel 用の seed runner を実行し、experiment.result_processing の関数で結果を整理します。

In [1]:
import sys
import time
from pathlib import Path

import jax
import jax.numpy as jnp

REPO_ROOT = Path.cwd().resolve()
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

jax.config.update("jax_enable_x64", True)
print(f"Project root: {REPO_ROOT}")

Project root: /Users/yanoshouta/dev/degenerate-diffusion/experiment


In [2]:
from dataclasses import dataclass
from typing import Any, Dict, Tuple

from sympy import Array, symbols

from degenerate_diffusion.evaluation.likelihood_evaluator import LikelihoodEvaluator
from degenerate_diffusion.processes.degenerate_diffusion_process import DegenerateDiffusionProcess


@dataclass
class FnModelSettings:
    model: Any
    evaluator: Any
    true_theta: Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]
    bounds_theta1: list[tuple[float, float]]
    bounds_theta2: list[tuple[float, float]]
    bounds_theta3: list[tuple[float, float]]
    t_max: float
    h: float
    burn_out: float
    dt: float
    loop_plan: Dict[int, tuple[str, str, str]]
    initial_theta_stage0: Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]


# --- FNmodel 定義（build_seed_runner_demo.ipynb と同一設定）
x_sym, y_sym = symbols("x, y")
theta_10, theta_20, theta_21 = symbols("theta_10 theta_20 theta_21")
theta_30, theta_31 = symbols("theta_30 theta_31")

x = Array([x_sym])
y = Array([y_sym])
theta_1 = Array([theta_10])
theta_2 = Array([theta_20, theta_21])
theta_3 = Array([theta_30, theta_31])

A = Array([theta_20 * y_sym - x_sym + theta_21])
B = Array([[theta_10]])
H = Array([(y_sym - y_sym**3 - x_sym + theta_31) / theta_30])

fnmodel = DegenerateDiffusionProcess(
    x=x,
    y=y,
    theta_1=theta_1,
    theta_2=theta_2,
    theta_3=theta_3,
    A=A,
    B=B,
    H=H,
)

fnmodel_likelihood = LikelihoodEvaluator(fnmodel)

true_theta = (
    jnp.array([0.3]),
    jnp.array([1.5, 0.8]),
    jnp.array([0.1, 0.0]),
)

bounds_theta1 = [(0.1, 0.5)]
bounds_theta2 = [(0.5, 2.5), (0.5, 1.5)]
bounds_theta3 = [(0.01, 0.3), (-1.0, 1.0)]

loop_plan: Dict[int, tuple[str, str, str]] = {
    1: ("B", "B", "B"),
    2: ("M", "M", "M"),
    3: ("M", "M", "M"),
    4: ("M", "M", "M"),
}

initial_theta_stage0 = (
    jnp.array([0.2]),
    jnp.array([0.5, 0.5]),
    jnp.array([0.2, 0.1]),
)

settings = FnModelSettings(
    model=fnmodel,
    evaluator=fnmodel_likelihood,
    true_theta=true_theta,
    bounds_theta1=bounds_theta1,
    bounds_theta2=bounds_theta2,
    bounds_theta3=bounds_theta3,
    t_max=100.0,
    h=0.05,
    burn_out=50.0,
    dt=0.001,
    loop_plan=loop_plan,
    initial_theta_stage0=initial_theta_stage0,
)

settings

FnModelSettings(model=DegenerateDiffusionProcess(x=[x], y=[y], theta_1=[theta_10], theta_2=[theta_20, theta_21], theta_3=[theta_30, theta_31], A=SymbolicArtifact(expr=[theta_20*y + theta_21 - x], func=<function _lambdifygenerated at 0x115915080>), B=SymbolicArtifact(expr=[[theta_10]], func=<function _lambdifygenerated at 0x115915a80>), H=SymbolicArtifact(expr=[(theta_31 - x - y**3 + y)/theta_30], func=<function _lambdifygenerated at 0x1159168e0>)), evaluator=<degenerate_diffusion.evaluation.likelihood_evaluator.LikelihoodEvaluator object at 0x115857550>, true_theta=(Array([0.3], dtype=float64), Array([1.5, 0.8], dtype=float64), Array([0.1, 0. ], dtype=float64)), bounds_theta1=[(0.1, 0.5)], bounds_theta2=[(0.5, 2.5), (0.5, 1.5)], bounds_theta3=[(0.01, 0.3), (-1.0, 1.0)], t_max=100.0, h=0.05, burn_out=50.0, dt=0.001, loop_plan={1: ('B', 'B', 'B'), 2: ('M', 'M', 'M'), 3: ('M', 'M', 'M'), 4: ('M', 'M', 'M')}, initial_theta_stage0=(Array([0.2], dtype=float64), Array([0.5, 0.5], dtype=float6

In [3]:
import importlib
import degenerate_diffusion.estimation.parameter_estimator as parameter_estimator_mod
import degenerate_diffusion.estimation.loop_estimation_algorithm as loop_algo_mod
from result_processing import process_and_save


# reload to pick up local edits when experimenting


parameter_estimator_mod = importlib.reload(parameter_estimator_mod)


loop_algo_mod = importlib.reload(loop_algo_mod)


SeedRunnerConfig = loop_algo_mod.SeedRunnerConfig


build_seed_runner = loop_algo_mod.build_seed_runner


seed_runner_config = SeedRunnerConfig(
    true_theta=settings.true_theta,
    t_max=settings.t_max,
    h=settings.h,
    burn_out=settings.burn_out,
    dt=settings.dt,
    bounds_theta1=settings.bounds_theta1,
    bounds_theta2=settings.bounds_theta2,
    bounds_theta3=settings.bounds_theta3,
    newton_kwargs={},
    nuts_kwargs={},
    one_step_kwargs={},
)


seed_runner = build_seed_runner(
    evaluator=settings.evaluator,
    model=settings.model,
    plan=settings.loop_plan,
    config=seed_runner_config,
)


vectorized_seed_runner = jax.jit(jax.vmap(seed_runner, in_axes=(0, None)))

In [4]:
seeds = jnp.arange(400, dtype=jnp.int32)
start = time.perf_counter()


theta10_batch, theta1_batch, theta2_batch, theta3_batch = vectorized_seed_runner(
    seeds,
    settings.initial_theta_stage0,
)


elapsed = time.perf_counter() - start


theta10_batch, theta1_batch, theta2_batch, theta3_batch = jax.device_get(
    (theta10_batch, theta1_batch, theta2_batch, theta3_batch),
)


print(f"Processed {len(seeds)} seeds in {elapsed:.3f} seconds.")

Processed 400 seeds in 26.240 seconds.


In [7]:
batches = {
    "theta10": theta10_batch,
    "theta1": theta1_batch,
    "theta2": theta2_batch,
    "theta3": theta3_batch,
}


df_results, summary_table, output_path = process_and_save(
    seeds,
    batches,
    output_path=Path("experiment/fnmodel_seed_runner_results.csv"),
)


print(f"Saved full results to {output_path.resolve()}")


summary_table

Saved full results to /Users/yanoshouta/dev/degenerate-diffusion/experiment/experiment/fnmodel_seed_runner_results.csv


,parameter,mean,std
0,theta10_0_1,0.337541,0.027923
1,theta10_0_2,0.301035,0.004953
2,theta10_0_3,0.300306,0.004823
3,theta1_0_1,0.378477,0.012102
4,theta1_0_2,0.304198,0.004315
5,theta1_0_3,0.309004,0.004190
6,theta2_0_1,1.461415,0.058799
7,theta2_0_2,1.494720,0.059966
8,theta2_0_3,1.503372,0.060667
9,theta2_1_1,0.775452,0.045795


In [1]:
summary_table.to_csv(Path("experiment/fnmodel_summary_table.csv"), index=False)

NameError: name 'summary_table' is not defined

In [3]:
from result_processing import build_theta_latex_table
latex = build_theta_latex_table(
    "experiment/fnmodel_summary_table.csv",
    {
        "theta10": [0.3],
        "theta1": [0.3],
        "theta2": [1.5, 0.8],
        "theta3": [0.1, 0.0],
    },
)
print(latex)

\begin{tabular}{ccccccc}
\hline
$k$ & $\theta^{{k,0}}_{1,0}$ & $\theta^{{k}}_{1,0}$ & $\theta^{{k}}_{2,0}$ & $\theta^{{k}}_{2,1}$ & $\theta^{{k}}_{3,0}$ & $\theta^{{k}}_{3,1}$ \
\hline
true & 0.300 & 0.300 & 1.500 & 0.800 & 0.100 & 0.000 \\
1 & 0.338 (0.028) & 0.378 (0.012) & 1.461 (0.059) & 0.775 (0.046) & 0.104 (0.000) & 0.001 (0.001) \\
2 & 0.301 (0.005) & 0.304 (0.004) & 1.495 (0.060) & 0.796 (0.047) & 0.101 (0.000) & 0.000 (0.000) \\
3 & 0.300 (0.005) & 0.309 (0.004) & 1.503 (0.061) & 0.802 (0.047) & 0.100 (0.000) & 0.000 (0.000) \\
\hline
\end{tabular}
